In [1]:
import requests

/Users/calvinpineda/Desktop/python-projects/my-web-scraping-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import xml.etree.ElementTree as ET

In [3]:
from bs4 import BeautifulSoup

In [4]:
import time

In [5]:
import random

In [6]:
# lets check if my XML has any missing tags

In [6]:
def check_xml(file_path):
    try:
        ET.parse(file_path)
        print('XML file contains no missing tags')
    except ET.ParseError:
        print('XML file contains missing tags - please fix')

In [7]:
check_xml('/Users/calvinpineda/Downloads/sitemap-web-air.xml')

XML file contains no missing tags


In [6]:
def find_urls(file_path, namespaces):
    oci_url_list = []
    tree = ET.parse(file_path)
    root = tree.getroot()
    for element in root:
        loc_element = element.find('ns:loc', namespaces=namespaces)
        if loc_element is not None:
            oci_url = loc_element.text
            oci_url_list.append(oci_url)
    return oci_url_list

In [7]:
def find_keyword(oci_url_list, keyword, max_urls=None): # max_url is used for testing
    store_bad_url = []  # List to store URLs with bad responses
    oci_url_list_contains_keyword = []  # List to store URLs that contain the keyword
    processed_urls = oci_url_list[:max_urls] if max_urls else oci_url_list # run up to max_url, else, run entire list
    for oci_url in processed_urls:
        try:
            response_url = requests.get(oci_url)
            if response_url.status_code == 200:
                url_text = response_url.text
                url_text_formatting = BeautifulSoup(url_text, 'html5lib')
                url_text_get = url_text_formatting.get_text().lower()
                if keyword in url_text_get:
                    url_count = url_text_get.count(keyword)
                    oci_url_list_contains_keyword.append((oci_url, url_count))
                    print(f"Processed {oci_url} - Keyword found {url_count} times")
                else:
                    print(f"Processed {oci_url} - Keyword not found")
            else:
                store_bad_url.append((oci_url, response_url.status_code))
                print(f"Processed {oci_url} - Failed with status: {response_url.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error processing {oci_url}: {e}")
        time.sleep(random.uniform(30, 45))
    return oci_url_list_contains_keyword, store_bad_url

In [10]:
def master_function():
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}  # Adjust the namespace as needed
    oci_url_list = find_urls('/Users/calvinpineda/Downloads/sitemap-web-air.xml', namespace)  # Correct path
    keyword = "Airfare".lower()  # Specify your keyword
    oci_url_list_containing_keywords, stored_bad_url = find_keyword(oci_url_list, keyword)
    print(oci_url_list_containing_keywords)  # Optionally print the results

In [11]:
master_function()

Processed https://www.oceaniacruises.com - Keyword found 4 times
Processed https://www.oceaniacruises.com/2024-voyage-collection - Keyword found 1 times
Processed https://www.oceaniacruises.com/2025-around-the-world-cruise - Keyword found 1 times
Processed https://www.oceaniacruises.com/2026-around-the-world-cruise - Keyword found 1 times
Processed https://www.oceaniacruises.com/air - Keyword found 3 times
Processed https://www.oceaniacruises.com/concierge-veranda - Keyword found 4 times
Processed https://www.oceaniacruises.com/grand-voyage-cruises - Keyword found 2 times
Processed https://www.oceaniacruises.com/oceania-club-benefits - Keyword not found
Processed https://www.oceaniacruises.com/ships/allura - Keyword found 1 times
Processed https://www.oceaniacruises.com/ships/insignia - Keyword found 3 times
Processed https://www.oceaniacruises.com/ships/marina - Keyword found 3 times
Processed https://www.oceaniacruises.com/ships/nautica - Keyword found 3 times
Processed https://www.o

In [41]:
url_24 = 'https://www.oceaniacruises.com/2024-voyage-collection'

In [43]:
response_24 = requests.get(url_24)

In [8]:
url_24 = 'https://www.oceaniacruises.com/2024-voyage-collection'
response_24 = requests.get(url_24)
if response_24.status_code == 200:
    soup_24 = BeautifulSoup(response_24.content, 'html5lib')
    keyword_24 = 'Airfare'.lower()
    text_content = soup_24.get_text().lower()
    with open ('page_content.txt', 'w', encoding='utf-8') as file:
        file.write(text_content)
    print('text content saved to txt file')
    
else:
    print(f"Failed to fetch the page. Status code: {response_24.status_code}")

text content saved to txt file


In [2]:
# peform the same action, wthout beautiful soup
url_25 = 'https://www.oceaniacruises.com/2024-voyage-collection'
response_25 = requests.get(url_25)
if response_25.status_code == 200:
    webpage_content = response_25.text.lower()
    with open ('page_content_request_only.txt', 'w', encoding='utf-8') as file:
        file.write(webpage_content)
    print ('text content saved to txt file')
else:
    print(f"failed to fetch the page, status code: {response_25.status_code}")

text content saved to txt file
